In [1]:
# IMPORTATIONS
import json
import logging
import sys
import os
from typing import List
from degiro_connector.trading.api import API as TradingAPI
from degiro_connector.trading.models.trading_pb2 import Credentials, ProductSearch, ProductsInfo
import shelve
from degiro_connector.quotecast.api import API as QuotecastAPI
from degiro_connector.quotecast.actions.action_get_chart import ChartHelper
from degiro_connector.quotecast.models.quotecast_pb2 import Chart
import pandas as pd
from datetime import datetime
#from datetime import date
from datetime import timedelta
import traceback
import numpy as np
from scipy import stats
from scipy.interpolate import interp1d
import re
from multiprocessing import  Pool
from functools import partial
import yfinance as yf
import threading, time, random
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
import math
import itertools

class cachedApi:
    def __init__(self, file:str, credentials=Credentials):
        self.__db = shelve.open(file)
        self.__trading_api = TradingAPI(credentials=credentials)
        self.__user_token = None
        self.__quotecast_api = None
        self.mutex = threading.Lock()
        
    def logout(self):
        self.__trading_api.logout()
    
    def cache_get(self, k):
        r = None
        self.mutex.acquire()
        try:
            r = self.__db[k]
        except:
            None
        self.mutex.release()
        return r

    def cache_set(self, k,v):
        self.mutex.acquire()
        self.__db[k] = v
        self.mutex.release()
    
    def get_config(self):
        return self.__trading_api.credentials

    def get_config(self,**kwargs):
        k = 'get_config' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_config(**kwargs)
            self.cache_set(k,r)
        #print(r)
        self.__user_token = r['clientId']
        #print(f"token:{self.__user_token}")
        return r

    def get_client_details(self,**kwargs):
        k = 'get_client_details' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_client_details(**kwargs)
            self.cache_set(k,r)
        #print(r)
        self.__trading_api.credentials.int_account = r["data"]["intAccount"]
        #print(f"intAccount:{self.__trading_api.credentials.int_account}")
        return r
    
    def connect(self):
        self.__trading_api.connect()
        if not self.__user_token:
            self.get_config()
        if self.__user_token:
            self.__quotecast_api = QuotecastAPI(user_token=self.__user_token)   
        session_id = self.__trading_api.connection_storage.session_id
        #print("You are now connected, with the session id :", session_id)

    def get_list_list(self):
        return self.__trading_api.get_favourites_list(raw=True)

    def create_favourite_list(self,**kwargs):
        return self.__trading_api.create_favourite_list(**kwargs)
    
    def delete_favourite_list(self,**kwargs):
        return self.__trading_api.delete_favourite_list(**kwargs)
    
    def put_favourite_list_product(self,**kwargs):
        return self.__trading_api.put_favourite_list_product(**kwargs)
    
    def get_products_config(self,**kwargs):
        k = 'get_products_config' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_products_config(**kwargs)
            self.cache_set(k,r)
        self.indices = {}
        for li in r['indices']:
            self.indices[li['id']] = DictObj(li)
        self.countries = {}
        for li in r['countries']:
            self.countries[li['id']] = DictObj(li)
        self.exchanges = {}
        for li in r['exchanges']:
            self.exchanges[li['id']] = DictObj(li)      
        self.stockCountries =  r['stockCountries']
        return r
     
    def get_company_ratios(self,**kwargs):
        k = 'get_company_ratios' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_company_ratios(**kwargs)
            self.cache_set(k,r)
        try:
            codes = {}
                
            if 'data' in r and 'currentRatios' in r['data'] and 'ratiosGroups' in r['data']['currentRatios']:
                for an in r['data']['currentRatios']['ratiosGroups']:
                    for i in an['items']:
                        v = i.get('value') or np.NaN  # value
                        t = i.get('type') or None # type of parameter
                        k = i.get('id') or None # name of parameter
                        m = i.get('name') or "" # meaning
                        if t == 'N' and not pd.isna(v): v = float(v)
                        #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                        if not m.__contains__(' per '): v = v * 1#000000
                        if k:
                            codes[k] = { 'meaning':m, 'value':v }

            if 'data' in r and 'forecastData' in r['data'] and 'ratios' in r['data']['forecastData']:
                for i in r['data']['forecastData']['ratios']:
                    #print(i)
                    v = i.get('value') or np.NaN  # value
                    t = i.get('type') or None # type of parameter
                    k = i.get('id') or None # name of parameter
                    m = i.get('name') or "" # meaning
                    if t == 'N' and not pd.isna(v): v = float(v)
                    #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                    if not m.__contains__(' per '): v = v * 1#000000
                    if k:
                        codes[k] = { 'meaning':m, 'value':v }

            if 'data' in r and 'consRecommendationTrend' in r['data'] and 'ratings' in r['data']['consRecommendationTrend']:
                for i in r['data']['consRecommendationTrend']['ratings']:
                    #print(i)
                    v = i.get('value') or np.NaN  # value
                    k = ('ratings_'+i.get('periodType')) or None # name of parameter
                    if t == 'N' and not pd.isna(v): v = float(v)
                    #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                    if not m.__contains__(' per '): v = v * 1#000000
                    if k:
                        codes[k] = { 'meaning':'', 'value':v }
                    
            codes['priceCurrency'] = { 'meaning':'', 'value':r['data']['currentRatios']['priceCurrency'] }
            if len(codes['priceCurrency']) <= 1:
                codes['priceCurrency'] = { 'meaning':'', 'value':r['data']['currentRatios']['currency'] }
        except:
            None
        return codes

    def get_financial_statements(self,**kwargs):
        k = 'get_financial_statements' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.get_financial_statements(**kwargs)
            self.cache_set(k,r)
        codes_array = []
        if r:
            try:
                for t in ('annual','interim'):
                    if t in r['data']:
                        for an in r['data'][t]:
                                endDate = datetime.strptime(an.get('endDate'), '%Y-%m-%d')#T%H:%M:%S')
                                fiscalYear = an.get('fiscalYear')
                                periodNumber = an.get('periodNumber') or 'Y'
                                codes = {}
                                for st in an['statements']:
                                    periodLength = st.get('periodLength')
                                    periodType = st.get('periodType')
                                    for i in st['items']:
                                        v = i.get('value') or np.NaN 
                                        if not pd.isna(v): v = float(v)
                                        if not i.get('meaning').__contains__(' per '): v = v * 1#000000
                                        codes[i.get('code')] = { 'meaning':i.get('meaning'), 'value':v }
                                codes_array += [ codes ]
            except:
                #print(k)
                #traceback.print_exc()
                #del self.cache_get(k)
                None
        return codes_array
    
    
    
    def get_estimates_summaries(self,**kwargs):
        k = 'get_estimates_summaries_' + str(kwargs)
        r = self.cache_get(k)
            #print("get_estimates_summaries cache hit", type(r))
        if r is None:
            r = self.__trading_api.get_estimates_summaries(**kwargs)
            #print("get_estimates_summaries cache miss", type(r))
            self.cache_set(k,r)
        return r
    
    def get_products_info(self,**kwargs):
        k = 'get_products_info' + str(kwargs)
        r = self.cache_get(k)
            #print("get_products_info cache hit", r)
        if r is None:
            r = self.__trading_api.get_products_info(**kwargs)
            #print("get_products_info cache miss", r)
            self.cache_set(k,r)
        return r

    def get_chart(self,**kwargs):
        k = 'get_chart' + str(kwargs)
        r = self.cache_get(k)
            #print("get_chart cache hit", r)
        if r is None:
            r = self.__quotecast_api.get_chart(**kwargs)
            #print("get_chart cache miss", r)
            self.cache_set(k,r)
        return r
   
    def product_search(self,**kwargs):
        k = 'product_search' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            r = self.__trading_api.product_search(**kwargs)
#        if hasattr(r, 'products'):
            self.cache_set(k,r)
#        else:
#            r = None
        return r

    def get_longtermprice(self,vwdIdSecondary:str):
        qrequest = Chart.Request()
        qrequest.culture = "fr-FR"
        qrequest.period = Chart.Interval.P1Y
        qrequest.requestid = "1"
        qrequest.resolution = Chart.Interval.P1D
        qrequest.series.append("ohlc:"+vwdIdSecondary)
        qrequest.tz = "Europe/Paris"
        k = 'get_chart' + str(qrequest)
        chart = self.cache_get(k)
        if chart is None:
            chart = trading_api.get_chart(request=qrequest,raw=False)
            self.cache_set(k,chart)
        price = pd.DataFrame()
        try:
            c2=ChartHelper.format_chart(chart=chart, copy=False)
            price = ChartHelper.serie_to_df(serie=chart.series[0])
            price["timestamp"] = pd.to_datetime(price["timestamp"], unit="s")
        except:
            #print(f"Error chart {vwdIdSecondary}")
            None
        #price.set_index("timestamp", inplace=True)
        return price


    def get_company_profile(self,**kwargs):
        k = 'get_company_profile' + str(kwargs)
        r = self.cache_get(k)
        if r is None:
            #searching on Degiro
            r = self.__trading_api.get_company_profile(product_isin=kwargs['product_isin'], raw=kwargs['raw'])
            self.cache_set(k,r)
        
        codes = {}
        if r is not None and 'data' in r:
            r_data = r['data']
            try:
                codes['sector'] = r_data['sector']
            except:
                None
            try:
                codes['industry'] =  r_data['industry']
            except:
                None
            try:
                codes['country'] =  r_data['contacts']['COUNTRY']
            except:
                None
            try:
                codes['floatShares'] = float(r_data['shrFloating']) / 10**6
            except:
                None
            try:
                codes['businessSummary'] = r_data['businessSummary']
            except:
                None
   
            try:
                if 'ratios' in r_data and 'ratiosGroups' in r_data['ratios']:
                    for an in r_data['ratios']['ratiosGroups']:
                        for i in an['items']:
                            v = i.get('value') or np.NaN  # value
                            t = i.get('type') or None # type of parameter
                            k = i.get('id') or None # name of parameter
                            m = i.get('name') or "" # meaning
                            if t == 'N' and not pd.isna(v): v = float(v)
                            #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                            if not m.__contains__(' per '): v = v * 1#000000
                            if k:
                                codes[k] = { 'meaning':m, 'value':v }
                if 'forecastData' in r_data and 'ratios' in r_data['forecastData']:
                    for i in r_data['forecastData']['ratios']:
                        #print(i)
                        v = i.get('value') or np.NaN  # value
                        t = i.get('type') or None # type of parameter
                        k = i.get('id') or None # name of parameter
                        m = i.get('name') or "" # meaning
                        if t == 'N' and not pd.isna(v): v = float(v)
                        #elif t == 'D': v = datetime.strptime(v, '%Y-%m-%dT%H:%M:%S') #pd.to_datetime(v)
                        if not m.__contains__(' per '): v = v * 1#000000
                        if k:
                            codes[k] = { 'meaning':m, 'value':v }
            except:
                None
        else: 
            # searching on Yahoo! finance
            try:
                r = self.cache_get('Y_'+k)
            except:
                sym = yf.Ticker(kwargs['product_isin'])
                r = sym.info
                try:
                    r['marketCap'] /= 1000.0
                except:
                    pass
                self.cache_set('Y_'+k, r)
                print(f"OK from Yahoo {kwargs['product_isin']}")
            codes = r
        return codes


def isna(num):
    return num!= num    

def get(d,k):
    r = np.NaN # sys.float_info.epsilon #float("nan")np.NaN
    if d is not None and (type(d) is dict) and k in d:
        r = d[k]
        if (type(r) is dict) and ('value' in r):
            r = r['value']
    else:
        r = np.NaN
    return r

def getmin(d,a): # get min of defined parameter in array a
    r = 10**6
    isset = False
    for p in a:
        v = get(d, p)
        if v == v:
            r = min(r, v)
            isset = True
    if isset:
        return r
    else:
        return np.NaN
            
    
def yget(d,k):
    r = np.NaN # sys.float_info.epsilon #float("nan")np.NaN
    if d is not None and k in d:
        r=d[k]
        if (type(r) is dict) and ('value' in r):
            r = r['value']
    else:
        r = np.NaN
    try:
        r = float(r)
    except:
        r = str(r)
        if r == 'None' or r == '':
            r = np.NaN
    return r

   
def parallelize_dataframe(df, func, n_cores=os.cpu_count()):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def assess_map(product, eee):
    p = DictObj(dict(product))
    row = {}
    #if p.isin != 'PTGNV0AM0001': return row
    #print(p)
    try:
        company_profile = trading_api.get_company_profile(product_isin=p.isin, raw=True)
        row['symbol'] = p.symbol
        row['isin'] = p.isin
        row['id'] = p.id
        row['vwdId'] = f'{p.vwdIdentifierType}:{p.vwdId}' if hasattr(p, 'vwdId') and hasattr(p, 'vwdIdentifierType') else ''
        row['vwdIdSecondary'] = f'{p.vwdIdentifierTypeSecondary}:{p.vwdIdSecondary}' if hasattr(p, 'vwdIdSecondary') and hasattr(p, 'vwdIdentifierTypeSecondary') else ''
        
        row['name'] = p.name.upper()
        row['sector'] =   yget(company_profile, 'sector')  
        row['industry'] = yget(company_profile, 'industry')
        '''
        desc = yget(company_profile, 'businessSummary')
        if desc is not None and type(desc) == str:
            if re.match(r'.*REIT.*data.?center.*', desc):
                row['industry'] += '!!'
                print(row['name'])
        '''
        if isinstance(row['industry'], str): row['industry'] = row['industry'].replace(' (NEC)', '')
        row['country'] = yget(company_profile, 'country') 
        row['eee'] = 1 if row['country'] in eee else 0
        
        row['volume'] = get(company_profile,"VOL10DAVG") or yget(company_profile, 'volume')
        #row['volume'] = row['volume'] / 1000 if not pd.isna(row['volume'])
        row['marketCap'] = get(company_profile,"MKTCAP") or yget(company_profile, 'marketCap')  
        if not pd.isna(row['marketCap']):
            row['marketCap'] /= 1000
        row['closePrice'] = p.closePrice if hasattr(p, 'closePrice') else 0 # price shown on screen, depends on the stock exchange
        row['Cur'] = p.currency if hasattr(p, 'currency') else '' # currency shown on screen
        if row['Cur'] == 'EUR':
            row['sortCur'] = 3 # sortCur is a temp column used to determine which exchange to keep when a stock is listed on different place. Higher value means more change to be selected
        elif row['Cur'] == 'USD':
            row['sortCur'] = 1
        else:
            row['sortCur'] = 0

        codes = trading_api.get_company_ratios(product_isin=p.isin, raw=True)
        #display(codes)
        row['StmPrice'] = get(codes,"NPRICE") # price used to compute statements, ratio, etc
        row['StmCur'] = get(codes,"priceCurrency") # currency used to compute market capitalization, statements, ratio, etc
        row['Δ1Y%'] = get(codes,"PR52WKPCT") # price diff 1 year
        row['Δ13W%'] = get(codes,"PR13WKPCT") # price diff 3 months
        row['Δ1W%'] = get(codes,"PR5DAYPRC") # price diff 1 week
        row['Δ1D%'] =get(codes,"PR1DAYPRC")

        if row['vwdId']:
            df = trading_api.get_longtermprice(row['vwdId'])
            df = df[-200:]
            mean200D = np.NaN
            if df.shape[0] == 200:
                mean200D = df['close'].mean()
                row['%M200D'] = (row['closePrice'] - mean200D) / mean200D
            elif df.shape[0] == 0 and row['vwdIdSecondary']:
                df = trading_api.get_longtermprice(row['vwdIdSecondary'])
                df = df[-200:]
                mean200D = np.NaN
                if df.shape[0] == 200:
                    mean200D = df['close'].mean()
                    row['%M200D'] = (row['closePrice'] - mean200D) / mean200D
        if not '%M200D'in row:
            row['%M200D'] = np.NaN
            #print(f"Cannot get 200-day chart data for {p.name}")
        
        h = get(codes,"NHIG")
        l = get(codes,"NLOW")
        row['L%H'] = int(100*(row['StmPrice'] - l)/(h-l)) if not pd.isna(h) and not pd.isna(l) and not pd.isna(row['StmPrice']) and h>l else np.NaN

        row['Yield'] =  get(codes,'YLD5YAVG') # dividend yield
        row['Payout'] =  get(codes,'TTMPAYRAT') # dividend yield
        row['β'] = get(codes,"BETA") # correlation 
        row['Reco'] = get(codes,'ratings_CURR') # current recommendation
        row['ΔFOCF5'] = get(codes,'FOCF_AYr5CAGR') # increase of free operational cash flow, 5Y CAGR
        row['P/FCF'] = get(codes,'TTMPRFCFPS') # Price to Free Cash Flow per Share - trailing 12 months

        a = get(codes,'TTMFCF') # Free Cash Flow - trailing 12 month
        b = get(codes,'TTMNIAC') # Net Income available to common - trailing 12 months
        row['FCF/NI'] = a/b if not pd.isna(a) and not pd.isna(b) and b > 0 else np.NaN

        
        row['ΔREV'] = get(codes,"REVCHNGYR")  # Revenue Change % - most recent quarter 1 year ago
        row['ΔREV3'] = get(codes,"REVGRPCT")  # "Growth rate% -  Revenue, 3 year";
        row['ΔREV5'] = get(codes,"REVTRENDGR")  # "Revenue growth rate, 5 year"; REVPS5YGR "Revenue/share (5 yr growth)"; -- should be > 0
        if pd.isna(row['ΔREV5']):
            row['ΔREV5'] = yget(company_profile, 'revenueGrowth')
        row['ΔΔREV1-3'] = row['ΔREV'] - row['ΔREV3']
        row['ΔΔREV3-5'] = row['ΔREV3'] - row['ΔREV5'] 
        
        row['ΔNPM'] = get(codes,"TTMNPMGN") # Net Profit Margin % - trailing 12 month"
        row['ΔNPM5'] = get(codes,"NPMTRENDGR") # "Net Profit Margin growth rate, 5 year"; -- should be > 0
        row['ΔΔNPM1-5'] = row['ΔNPM'] - row['ΔNPM5'] 
        
        row['ΔEPS'] = get(codes,'TTMEPSCHG')   # latest "Growth rate% - EPS, TTM";
        if pd.isna(row['ΔEPS']):
            row['ΔEPS'] = yget(company_profile, 'earningsGrowth')
        row['ΔEPS3'] = get(codes,"EPSGRPCT")   # "EPS Growth rate % - , 3 year CAGR";
        row['ΔEPS5'] = get(codes,"EPSTRENDGR") # "EPS growth rate %, 5 year CAGR";

        row['ΔΔEPS1-3'] = row['ΔEPS'] - row['ΔEPS3']
        row['ΔΔEPS3-5'] = row['ΔEPS3'] - row['ΔEPS5'] 
        row['ROEpct'] = get(codes,'TTMROEPCT')           # Return on average equity - trailing 12 month -- should be >20%
        if pd.isna(row['ROEpct']):
            row['ROEpct'] = yget(company_profile, 'returnOnEquity')
        row['ROE5Ypct'] = get(codes,'AROE5YAVG')         # Return on average equity avg 5Y -- should be >20%
        
        row['ΔROE'] = row['ROEpct'] - row['ROE5Ypct'] 
        
        row['P2TB'] = get(codes,'APR2TANBK')             # price to tangible book
        #row['dP2TB'] = get(codes,'BVTRENDGR')           # growth of price to tangible book, 5Y CAGR
        row['P2B'] = get(codes,'APRICE2BK')              # price to  book
        if pd.isna(row['P2B']):
            row['P2B'] = yget(company_profile, 'priceToBook')
        #row['dP2B'] = get(codes,'TanBV_AYr5CAGR')       # growth of P25B, 5Y
        row['PCF'] =  get(codes,'TTMPRCFPS')             # "Price to Cash Flow per share, near 1 idealy

        row['PE'] = get(codes,'PEINCLXOR')               #  P/E including extraordinary items - TTM - should be <50%
        if pd.isna(row['PE']):
            row['PE'] = yget(company_profile, 'trailingPE')
        row['fPE'] = get(codes,'ProjPE')          # forward PE
        if pd.isna(row['fPE']):
            row['fPE'] = yget(company_profile, 'forwardPE')

        row['PEG'] = row['PE'] / row['ΔEPS3'] if row['ΔEPS3'] and row['ΔEPS3']>0 else np.NaN # PEG ratio, should be <1
        if pd.isna(row['PEG']):
            row['PEG'] = yget(company_profile, 'pegRatio')
        row['fPEG'] = row['fPE'] / row['ΔEPS3']  if row['ΔEPS3'] and row['ΔEPS3']>0 else np.NaN    # forward PEG ratio, should be <1

        row['PS'] = get(codes,'TTMPR2REV')               #  Price to sales - trailing 12 month  -- should be between 2 to 4
        if pd.isna(row['PS']):
            row['PS'] = yget(company_profile, 'priceToSalesTrailing12Months')
        row['fPS'] = get(codes,'Price2ProjSales') # forward PS -- should be 2 to 4/
            
        row['%DEBT'] =  get(codes,'QTOTD2EQ') #"Total debt/total equity, percent, should be <100%
        row['QLTD2EQ'] =  get(codes,'QLTD2EQ') #"Total debt/total equity, percent, should be <100%
        row['QCURRATIO'] =  get(codes,'QCURRATIO') #"Total debt/total equity, percent, should be <100%
        if pd.isna(row['%DEBT']):
            row['%DEBT'] = yget(company_profile, 'debtToEquity')
        row['%DEBT'] = round(row['%DEBT']) if not pd.isna(row['%DEBT']) else np.NaN
        row['BV'] =  get(codes,'QBVPS') # QTANBVPS
        if pd.isna(row['BV']):
            row['BV'] =  get(codes,'ABVPS') # ATANBVPS
        if pd.isna(row['BV']):
            row['BV'] =  get(codes,'QTANBVPS')  
        if pd.isna(row['BV']):
            row['BV'] =  get(codes,'ATANBVPS')  
        if pd.isna(row['BV']):
            row['BV'] =  yget(company_profile, 'bookValue')
        row['BV'] =  (1 * (row['BV'] - row['StmPrice']) / row['StmPrice']) if not pd.isna(row['BV']) and not pd.isna(row['StmPrice']) and row['BV'] > 0 and row['StmPrice'] > 0 else np.NaN 
        
        # book value tangible / share price, last quarter >100% is fair

        # ratio : intrinsic value from free cash flow per share / price per share - should be > 100%
        # gain of free cash flow CAGR5Y is not available on DEGIRO, so I consider "free operational" cash flow
        dFOCF =  get(codes,'FOCF_AYr5CAGR') # gain of  free operational cash flow, CAGR 5 year.
        FCFS = get(codes,'TTMFCFSHR')   # free Cash Flow per share  - trailing 12 month
        row['IV'] = FCFS*((1-((1+dFOCF/100)*0.85)**10)/(1-((1+dFOCF/100)*0.85))+10*(((1+dFOCF/100)*0.85)**10)) if not pd.isna(dFOCF) and not pd.isna(FCFS) else np.NaN
        row['IV'] = (1*(FCFS-row['StmPrice'])/row['StmPrice']) if not pd.isna(row['StmPrice']) and row['StmPrice']>0 and not pd.isna(row['IV']) else np.NaN
        
        try:
            # 1/ EPS
            eps = get(codes,"TTMEPSINCX")   # "EPS including extraordinary items - trailing 12 month";
            if not pd.isna(eps):
                # 2/ growth rate min des 2 là, ou ΔEPS5?
                gr = getmin(codes,["REVTRENDGR", "TanBV_AYr5CAGR"]) #, "EPSTRENDGR", "EPSGRPCT"
                # 3/ projPE ou le double du précédent, min
                if not pd.isna(row['fPE']):
                    ppe = min (row['fPE'], 2* gr)
                else:
                    ppe = 2* gr
                fsv = eps*((1+gr/100)**5)*ppe/2                        
                row['FV'] = (fsv-row['StmPrice'])/row['StmPrice'] # 0=> stock price will double in 5 years          
        except:
            pass

        #dFOCF 	FCFS
        #BVS = get(codes,'ABVPS') #Book value (Total Equity) per share - most recent fiscal year
        #FCF = p.StmPrice / get(codes,'TTMPRFCFPS') # Price to Free Cash Flow per Share - trailing 12 months" 
        #dREV3 = get(codes,"REVGRPCT") #"Growth rate% -  Revenue, 3 year";
        #dBVS5 = get(codes,"BVTRENDGR") #"Book value per share growth rate, 5 year";
        #dTBE5 = get(codes,"TanBV_AYr5CAGR") #"Tangible Book Value, Total Equity, 5 Year CAGR";
        #dCSP5 = get(codes,"CSPTRENDGR") # "Capital Spending growth rate, 5 year";      
        #row['EV/EBITD'] = EV/EBITD if EV and EBITD and EBITD>0 else 0

    except:
        print(f"Error profile {p.symbol}")
        traceback.print_exc()
    return row    

def myassess(country, stock_list, info_df):
    eee = {}
    try:
        eee = {k:1 for k in pd.read_csv("eee.csv", header=None).T.values[0]}
    except:
        pass
    try:
        if hasattr(stock_list, 'products'):
            with ThreadPoolExecutor(max_workers = os.cpu_count()) as executor:
                results = executor.map(assess_map, stock_list.products, itertools.repeat(eee))
            for row in results:
                info_df = info_df.append(row,ignore_index=True)
        else:
            print("Stock market as no product", country)
        #info_df = info_df.astype({'FV':'Int64', 'IV':'Int64', 'BV' :'Int64','%DEBT' :'Int64','L%H':'Int64','eee':'Int64'})
    except:
        traceback.print_exc()
        pass
    return info_df

class DictObj:
    def __init__(self, in_dict:dict):
        assert isinstance(in_dict, dict)
        for key, val in in_dict.items():
            if isinstance(val, (list, tuple)):
               setattr(self, key, [DictObj(x) if isinstance(x, dict) else x for x in val])
            else:
               setattr(self, key, DictObj(val) if isinstance(val, dict) else val)

#############            
            
logging.basicConfig(level=logging.FATAL) #

username = os.getenv("GT_DG_USERNAME") or ""
password = os.getenv("GT_DG_PASSWORD") or ""

if username == "" or password == "":
    exit(0)
    
credentials = Credentials(
    int_account=None, # updated by get_client_details()
    username=username,
    password=password,
)


trading_api = cachedApi('/home/fab/GamestonkTerminal/.cachedb',credentials)
trading_api.connect()

try:
    # get all product list, countries, marketplaces
    products_config_dict = trading_api.get_products_config(raw=True)
    # get IntAccount
    trading_api.get_client_details()

    # this is the main dataframe that will be filled up
    info_df = pd.DataFrame()
    
    # stocked are browsed from counties(, and not marketplaces). This is the most reliable to get all stocks
    for li_dict in trading_api.stockCountries:
        li = DictObj(li_dict)
        stock_country_id = li.id
        country = trading_api.countries[li.country].name
        #if country != 'FR': continue
        # it's assumed that a country has less than 10x1000 stocks, so we browse up to 10 pages and stop once we got a partial page
        for page in range(0,10):
            request_stock = ProductSearch.RequestStocks(stock_country_id=stock_country_id,limit=1000,offset=page*1000,require_total=True)
            stock_list = trading_api.product_search(request=request_stock, raw=False)
            if hasattr(stock_list, 'products'):
                size = len(stock_list.products)
                print(f"country:{country} list:All ({size} stocks for page {page+1})")
                # dowload data for all stocks in the list. It's multi-thread even though the cache system is mono-thread...
                if stock_list: info_df = myassess(country, stock_list, info_df) 
                #for p in stock_list.products:
                #    assess_map(p, {})
                if size != 1000: break
            else:
                break
    print(f"Number of stock entries in all exchanges: {info_df.shape[0]}")
    info_df.set_index('id', inplace = True)
       
except Exception as e:
    print(e)
    print(repr(e))
    traceback.print_exc()

try:
    trading_api.logout()
except Exception as e:
    print(e)
    print(repr(e))
    traceback.print_exc()
 

country:FI list:All (169 stocks for page 1)
country:ES list:All (174 stocks for page 1)
country:HU list:All (31 stocks for page 1)
country:DK list:All (243 stocks for page 1)
country:PT list:All (42 stocks for page 1)
country:SE list:All (704 stocks for page 1)
country:CH list:All (235 stocks for page 1)
country:NO list:All (267 stocks for page 1)
country:NL list:All (131 stocks for page 1)
country:AT list:All (77 stocks for page 1)
country:PL list:All (407 stocks for page 1)
country:CZ list:All (38 stocks for page 1)
country:IT list:All (291 stocks for page 1)
country:SG list:All (189 stocks for page 1)
country:TR list:All (36 stocks for page 1)
country:BE list:All (149 stocks for page 1)
country:GR list:All (163 stocks for page 1)
country:FR list:All (725 stocks for page 1)
country:IE list:All (37 stocks for page 1)
country:HK list:All (785 stocks for page 1)
country:CA list:All (1000 stocks for page 1)
country:CA list:All (271 stocks for page 2)
country:GB list:All (1000 stocks for 

CRITICAL:degiro_connector.quotecast.actions.action_get_chart:'series'


country:US list:All (1000 stocks for page 4)
country:US list:All (1000 stocks for page 5)


CRITICAL:degiro_connector.quotecast.actions.action_get_chart:'series'
CRITICAL:degiro_connector.quotecast.actions.action_get_chart:'series'
CRITICAL:degiro_connector.quotecast.actions.action_get_chart:'series'
CRITICAL:degiro_connector.quotecast.actions.action_get_chart:'series'


country:US list:All (448 stocks for page 6)
country:AU list:All (1000 stocks for page 1)
country:AU list:All (55 stocks for page 2)
country:JP list:All (1000 stocks for page 1)
country:JP list:All (323 stocks for page 2)
country:DE list:All (1000 stocks for page 1)
country:DE list:All (1000 stocks for page 2)
country:DE list:All (1000 stocks for page 3)
country:DE list:All (1000 stocks for page 4)
country:DE list:All (1000 stocks for page 5)
country:DE list:All (1000 stocks for page 6)
country:DE list:All (1000 stocks for page 7)
country:DE list:All (1000 stocks for page 8)
country:DE list:All (1000 stocks for page 9)
country:DE list:All (350 stocks for page 10)
Number of stock entries in all exchanges: 24857


In [2]:
def var2rank(X,Y,x):
    r = 1 # default return if x is nan
    try:
        if x == x:
            y_interp = interp1d(x=X, y=Y,fill_value=(Y[0], Y[-1]), bounds_error=False)
            r = float(y_interp(x))
    except:
        traceback.print_exc()
        None
    return r

def var2quant(x,Q,name): 
    Y = Q.index.to_numpy()
    Y = (Y-Y[0])/(Y[-1]-Y[0])+1
    Y = np.asarray(Y)
    X=list(Q[name].to_dict().values())
    r = var2rank(X,Y,x[name])
    return r

def compute_rank(info_df, Q, k):
    info_df['score'] = 1
    for key, value in k.items():
        info_df['score'] *= info_df.apply(lambda x: var2quant(x,Q,key), axis = 1) ** value
    return info_df


######         

# compute quantile of all numeric parameters
Q = info_df.quantile(q=[0.1,0.25,0.5,0.75,0.9])

# per sector, correlation between 1YTTM gain and all other parameters. 
# Conglomerates are removed since result is the opposite from all other sector
# then we merge all sector by computing the mean of correlation factor for each parameter
#gainCorrelation=info_df.drop(info_df.index[ (info_df['Δ1Y%'] < Q['Δ1Y%'][0.75]) ])
gainCorrelation=info_df.groupby('sector').corrwith(info_df['Δ1Y%'],method ='spearman').mean(axis=0)
# Only some parameters are considered to compute the score
gainCorrelation = gainCorrelation[ [\
                                    "QLTD2EQ", "QCURRATIO", "Payout", "Yield", "β", \
                                    "Reco", "ΔFOCF5", "P/FCF", "FCF/NI", "ΔREV5", \
                                    'ΔREV', 'ΔREV3', 'ΔΔREV1-3', 'ΔΔREV3-5', 'ΔΔNPM1-5', \
                                    "ΔNPM5", "ΔNPM", "ΔEPS", "ΔEPS3", "ΔEPS5", \
                                    'ΔΔEPS1-3', 'ΔΔEPS3-5', "ROEpct", "ROE5Ypct", 'ΔROE', \
                                    "P2TB", "P2B", "PCF", "PE", "fPE", \
                                    "PEG", "fPEG", "PS", "fPS", "%DEBT" ] ]
# we normalize the power factors, we add the 1YTTM back in the table
gainMax=gainCorrelation.abs().mean()
gainCorrelation=gainCorrelation/gainMax
gainCorrelation=dict(gainCorrelation.sort_values(ascending = False))
gainCorrelation['Δ1Y%']=1

#display(gainCorrelation)

# compute the score for all stocks, using all CPU cores
if info_df.shape[0] > os.cpu_count():
    info_df = parallelize_dataframe(info_df,partial(compute_rank, Q=Q, k=gainCorrelation))
else:
    compute_rank(info_df, Q, gainCorrelation)

# we remove duplicates when a stock is listed on several exchanges.
info_df = info_df.sort_values(by=['name', 'sortCur', 'vwdId'], ascending = False).drop_duplicates(keep = 'first', subset = 'name')
info_df = info_df.sort_values(by=['isin', 'sortCur', 'vwdId'], ascending = False).drop_duplicates(keep = 'first', subset = 'isin')
print(f"Number of stock entries after removing duplicates: {info_df.shape[0]}")
info_df=info_df.sort_values(by='score', ascending=False)
#display(info_df[:30])


/home/fab/anaconda3/envs/gst/lib/python3.8/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


Number of stock entries after removing duplicates: 15359


In [3]:
def write2csv(df):
    now = datetime.now() # current date and time
    filename="degiro-export-"+now.strftime("%Y-%m-%d-%H-%M")+".csv"
    filepath='.'
    fullpath=os.path.join(filepath,filename)
    print(f"Writing csv file '{fullpath}' (encoding utf-8)")
    df.to_csv(fullpath, index=True, sep=str(';'), decimal=str(','),encoding='utf-8', )

####

write2csv(info_df)

Writing csv file './degiro-export-2022-04-20-09-32.csv' (encoding utf-8)


In [7]:
pd.set_option('display.max_colwidth', 25)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.2f}'.format

#display(info_df[:1])
split=10**(max(1,math.floor(math.log(info_df.shape[0],10))-1))
keeptop=math.floor(split*0.95)
cropsector=10#max(5,math.floor(split/50))
cropindustry=2#max(1,math.floor(cropsector/5))
print(f"Quantiles:{split}, keeping top {keeptop}th and above, limiting to {cropsector} stocks per sector and {cropindustry} per industry.")
df = info_df.copy()
df['score'] = pd.qcut(df['score'].rank(method='first'),q=split, retbins=False, labels=False)
df=df.drop(df.index[ ( (df['PE'] >Q['PE'][0.9]) | (df['%DEBT'] >Q['%DEBT'][0.9]) | (df['P2B'] >Q['P2B'][0.9]) | (df['Reco'] > Q['Reco'][0.5])  | (df['PEG'] > Q['PEG'][0.9])  )]) # | (df['PE'] >15) | (df['%DEBT'] >100) | (df['Δ1Y%'] < 8) |(df['P2TB'] >3) |(df['ΔREV5'] <5) |(df['ΔFOCF5'] <0) |(df['FV'] <-.50)  |(df['FCF/NI'] <0)
df=df.drop(df.index[ ( (df['score'] < keeptop) | (df['%M200D'] <0) | (df['L%H'] < 45) | (df['sector'] == "Financial") | (df['sector'] == "Transportation") )]) #| (df['Reco'] > 2.4) | (df['Δ1Y%'] < 20) | (df['FCF/NI'] <0) |(df['P2B'] >15) |(df['ΔREV'] <0) | (df['%DEBT'] >120)  | (df['industry'].str.contains("harma")) | (df['sector'].str.contains("inanc")) |(df['FV'] <-.35)      | (df['PE'] >15) | (df['%DEBT'] >100) | (df['Δ1Y%'] < 8) |(df['P2TB'] >3) |(df['ΔREV5'] <5) |(df['ΔFOCF5'] <0) |(df['FV'] <-.50)  |(df['FCF/NI'] <0)
df.drop([ 'sortCur', 'vwdId', 'vwdIdSecondary', 'StmCur', 'StmPrice', 'ΔΔEPS1-3', 'ΔΔEPS3-5',"ΔEPS3", "ΔEPS5","ROE5Ypct", 'ΔROE', 'ΔREV5', 'ΔREV3', 'ΔΔREV1-3', 'ΔΔREV3-5', 'ΔΔNPM1-5', "ΔNPM5",'QLTD2EQ','QCURRATIO'], axis=1, inplace=True)
df = df.sort_values(by=['industry','score'], ascending=False).groupby('industry').head(cropindustry).sort_values(by=['sector','score'], ascending=False).groupby('sector').head(cropsector).sort_values(by=['eee','Δ1W%','score'], ascending=False)#.sort_values(by=['eee','ROE5Ypct'], ascending=False) #'sector','industry','BV'
df = df.fillna("—")
print('Please read the readme.rtf to get the meaning of all the columns.')
print(f'Result has {df.shape[0]} lines')
display(df)

#write2csv(corr)
#import seaborn as sns
#sns.heatmap(corr)

Quantiles:1000, keeping top 950th and above, limiting to 10 stocks per sector and 2 per industry.
Please read the readme.rtf to get the meaning of all the columns.
Result has 54 lines


,symbol,isin,name,sector,industry,country,eee,volume,marketCap,closePrice,Cur,Δ1Y%,Δ13W%,Δ1W%,Δ1D%,%M200D,L%H,Yield,Payout,β,Reco,ΔFOCF5,P/FCF,FCF/NI,ΔREV,ΔNPM,ΔEPS,ROEpct,P2TB,P2B,PCF,PE,fPE,PEG,fPEG,PS,fPS,%DEBT,BV,IV,FV,score
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1177302,NEWA B,SE0000426546,NEW WAVE GROUP AB,Consumer Cyclical,Apparel & Accessories,Sweden (SWE),1.00,"166,140.00","8,459,745.00",181.40,SEK,122.30,21.91,22.57,-1.41,0.24,94.00,3.11,0.00,1.86,1.33,22.57,10.34,1.08,23.14,11.31,107.93,18.07,2.74,2.64,8.57,15.83,13.07,0.57,0.47,1.26,1.08,31.00,-0.62,-0.93,-0.59,967
1743668,ALMOU,FR0011033083,MOULINVEST SA,Basic Materials,Forest & Wood Products,France (FRA),1.00,"5,790.00","190,671.00",62.00,EUR,389.56,50.86,16.54,11.51,0.70,98.00,0.00,0.00,1.66,1.00,28.96,22.06,0.79,52.66,11.82,192.44,26.77,4.09,4.03,9.23,17.07,13.63,0.32,0.25,2.04,1.82,97.00,-0.75,-0.95,-0.34,996
5982194,SSH,DE0007346603,SUEDWESTDEUTSCHE SALZ...,Basic Materials,Agricultural Chemicals,Germany (DEU),1.00,10.00,"977,197.50",93.00,EUR,28.28,21.57,12.05,0.00,0.17,94.00,2.46,37.27,0.02,—,—,20.91,1.04,36.01,13.13,618.42,20.20,5.58,4.10,14.93,21.66,—,1.01,—,2.86,—,1.00,-0.76,—,-0.89,985
19747411,RHM,DE0007030009,RHEINMETALL AG,Consumer Cyclical,Engine & Powertrain S...,Germany (DEU),1.00,"312,040.00","9,164,782.00",211.40,EUR,138.01,135.56,7.26,0.05,1.10,98.00,2.53,36.83,0.90,1.69,25.65,21.87,1.07,2.54,7.64,53.13,18.16,5.56,3.77,13.76,31.24,19.32,3.74,2.31,1.62,1.37,38.00,-0.73,-0.95,-0.99,966
866154,GRVO,FR0000065971,GRAINES VOLTZ,Consumer/Non-Cyclical,Fishing & Farming,France (FRA),1.00,780.00,"247,944.40",167.50,EUR,42.55,-3.46,6.69,2.13,0.21,82.00,5.73,27.61,0.43,1.00,67.42,39.77,0.66,54.23,7.58,122.40,24.77,11.01,5.04,20.16,23.13,21.12,1.02,0.94,2.03,1.70,101.00,-0.80,-0.97,-1.02,999
1177361,POOL B,SE0000567539,POOLIA AB,Services,Employment Services,Sweden (SWE),1.00,"62,300.00","717,240.00",20.05,SEK,77.43,-3.14,5.53,5.75,0.27,91.00,4.55,44.38,1.54,—,99.25,9.00,0.99,31.49,4.24,753.28,43.22,3.63,4.40,6.53,11.60,13.83,0.14,0.16,0.38,0.35,13.00,-0.77,-0.91,0.44,988
855527,REC,BE0003656676,RECTICEL,Basic Materials,Commodity Chemicals,Belgium (BEL),1.00,"81,810.00","1,210,808.00",21.60,EUR,63.88,19.60,4.10,0.93,0.29,97.00,1.73,23.22,1.26,1.50,—,14.27,1.74,79.97,4.79,681.44,13.44,3.55,3.08,13.07,22.71,4.36,1.25,0.24,1.17,1.96,69.00,-0.68,—,-1.01,976
866029,ALDEL,FR0000054132,DELFINGEN INDUSTRY,Technology,"Auto, Truck & Motorcy...",France (FRA),1.00,720.00,"141,304.40",53.00,EUR,6.00,-2.57,3.92,-1.49,0.01,50.00,1.71,1.68,1.25,1.00,18.79,—,-0.02,130.50,16.94,"1,600.34",58.30,2.89,1.21,1.84,2.25,7.96,0.03,0.12,0.40,0.36,101.00,-0.08,-1.01,1.48,982
4796513,ANY,HU0000093257,ANY BIZTONSAGI NYOMDA...,Services,Specialized Printing ...,Hungary (HUN),1.00,"8,910.00","26,334,480.00","1,780.00",HUF,32.84,15.21,3.49,0.85,0.13,96.00,5.98,64.84,0.18,2.00,—,15.36,0.48,24.28,9.77,359.57,43.64,2.91,2.72,4.63,7.08,14.83,0.15,0.32,0.65,0.71,64.00,-0.63,—,0.62,984


In [5]:
def write2fav(df):
    if (df.shape[0] > 0):
        trading_api.connect()
        products_config_dict = trading_api.get_products_config(raw=True)
        #print(trading_api.get_client_details())
        now = datetime.now()
        prefix='Screener-'
        fl = trading_api.get_list_list()
        for l in fl['data']:
            if 'name' in l and l['name'].startswith(prefix):
                trading_api.delete_favourite_list(id=l['id'])
                print(f'Deleting DEGIRO favourite list \"{l["name"]}\"')
        name=prefix+now.strftime("%Y-%m-%d-%H-%M")
        print(f'Creating DEGIRO favourite list \"{name}\"')
        favorite_list_id = trading_api.create_favourite_list(name=name)
        for p in list(df.index[:50]):  
            # list is limited to 50 entries
            trading_api.put_favourite_list_product(id=favorite_list_id,product_id=p)
            #print(f'Adding product id {p}')
        trading_api.logout() 

write2fav(df)

Deleting DEGIRO favourite list "Screener-2022-04-20-08-36"
Creating DEGIRO favourite list "Screener-2022-04-20-09-32"
